In [1]:
from boardGPT.models import GameGPT

In [4]:
model, model_config = GameGPT.from_pretrained(repo_id="theartificialis/OthelloGPT-Synthetic-20m")

In [5]:
model

GameGPT(
  (token_emb_hook): HookPoint()
  (pos_emb_hook): HookPoint()
  (pre_logits_hook): HookPoint()
  (transformer): ModuleDict(
    (wte): Embedding(61, 512)
    (wpe): Embedding(60, 512)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-7): 8 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (qk_hook): HookPoint()
          (v_hook): HookPoint()
          (c_attn): Linear(in_features=512, out_features=1536, bias=False)
          (c_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=2048, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=2048, out_features=512, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    

In [ ]:
model.